## Import Statements

In [1]:
import gensim.downloader as api
import numpy as np
import re
import csv
import pandas as pd
import pprint
import string
import nltk
import sys
import spacy
import os
import pickle
    
from IPython.display import HTML
from nltk.corpus import wordnet 
from sklearn.manifold import TSNE
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize

pathToDatasets = '../datasets/'
pathToDataScripts = '../datasets/scripts/'
filePath = '../datasets/GoogleNews-vectors-negative300.bin'

sys.path.insert(0, pathToDataScripts)
from cleanDataset import tokenize_words, dataClean


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))

cleanDataset
Imported clean dataset!


## Downloading binaries and models


In [2]:
print("Should I reload the model?")
tstString = input()
if("no" in tstString.lower() or "n" in tstString.lower()):
    print(" didnt reload model! ")
else:
    print("loading the model!");
    if(os.path.exists(pathToDatasets+"word_vectors-300.pickle")):
        print("loading via pickle!")
        pickle_in = open(pathToDatasets+"word_vectors-300.pickle", "rb")
        word_vectors = pickle.load(pickle_in);
    else:
        print("Path didn't exist, therefore loading model!")
        word_vectors = api.load("glove-wiki-gigaword-300")
    nltk.download('vader_lexicon')
    nltk.download('punkt')
    nltk.download('averaged_perceptron_tagger')
    nltk.download('wordnet')
    print("Model Loaded!")

Should I reload the model?
yeet
loading the model!
loading via pickle!
Model Loaded!


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/ubuntu/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ubuntu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Global Variables and Global Objects

In [3]:
senty = SentimentIntensityAnalyzer()
vocabulary = word_vectors.vocab;

nlp = spacy.load('en')


NUMBER_OF_ALTERNATIVES = 5
TWEET_START = 1
NUM_OF_TWEETS = 30


VERBOSE_PRINTING = True
# VERBOSE_PRINTING = False

# USE_SPACY = False
USE_SPACY = True

COLOR_PRINTING = True
#COLOR_PRINTING = False

# PRINT_NEUTRAL = True
PRINT_NEUTRAL = False

PRINT_ALL_STRINGS = True
# PRINT_ALL_STRINGS = False

SHOW_ALTS = 30

punctuation = r"\"#$%&'+-/;<=>?@[\]*^_`{|}~"

## Class for Sentences


In [4]:
from SentenceClass import *

Imported class file!


## Utility Code

In [5]:
def printStrings(sentenceObj):
    
    numberOfPrints = 0
    newStrings = generateHTMLObjectsFromSentence(sentenceObj)
    mainSentiment = sentenceObj.ogSentiment;
    listOfSentencesWithSentiments = []
    bestSentiment = -sys.maxsize - 1
    worstSentiment = sys.maxsize
    
    bestSentimentString = "";
    worstSentimentString = "";
    
    
    for ind, tSentence in enumerate(newStrings):
            alteredTweet = tSentence.getSentence()
            htmlText = tSentence.getHTML()
            sentimentOfNewString = senty.polarity_scores(alteredTweet)['compound']
            newObj = SentenceWithSentiment(alteredTweet, sentimentOfNewString, htmlText)
            listOfSentencesWithSentiments.append(newObj)
            
            if(sentimentOfNewString >= bestSentiment):
                bestSentiment = sentimentOfNewString
                bestSentimentString = htmlText;
            elif(sentimentOfNewString < worstSentiment):
                worstSentiment = sentimentOfNewString
                worstSentimentString = htmlText;
                
            
            if(sentimentOfNewString == mainSentiment or sentimentOfNewString == 0.0):
                if(PRINT_ALL_STRINGS and numberOfPrints <= SHOW_ALTS): displayText("{0}: {1}".format(htmlText,sentimentOfNewString),'black')
                numberOfPrints+=1;
            elif(sentimentOfNewString > mainSentiment):
                if(PRINT_ALL_STRINGS and numberOfPrints <= SHOW_ALTS): displayText("{0}: {1}".format(htmlText,sentimentOfNewString),'green')
                numberOfPrints+=1;
            elif(sentimentOfNewString < mainSentiment and sentimentOfNewString != 0.0):
                if(PRINT_ALL_STRINGS and numberOfPrints <= SHOW_ALTS): displayText("{0}: {1}".format(htmlText,sentimentOfNewString),'red')
                numberOfPrints+=1;
                
    if(numberOfPrints > SHOW_ALTS): print("--- More options (total: {0}) possible, but not printed ---".format(numberOfPrints));
    if (worstSentimentString != ""): displayText("Worst Sentence: {0} : {1}".format(worstSentimentString, worstSentiment), color='red')
    if (bestSentimentString != ""): displayText("Best Sentence: {0} : {1}".format(bestSentimentString, bestSentiment), color='green') 
    
    
    return listOfSentencesWithSentiments;

def cstr(s, color='black', italics=False):
    if(COLOR_PRINTING):
        if(italics):
            return cstr("<i>{0}</i>".format(s), color);
        return "<text style=color:{}>{}</text>".format(color, s)
    else:
        return "{}".format(s)

def displayText(text, color='black'):
    if(COLOR_PRINTING):
        display(HTML(cstr(text, color)));
        return
    print("{}".format(text));
    
    
def cleanAndTokenizeText(text):
    text = text.lower();
    newString = ""
    for char in text:
        if char not in punctuation:
            newString += char
    text = word_tokenize(newString)
    return text;

def getPOSTags(tweet):
    if(USE_SPACY == False):
        tags = nltk.pos_tag(tweet)
        return tags;    
    tweet = ' '.join(tweet)
    doc = nlp(tweet)
    tags = [(token.text, token.tag_) for token in doc] # since the format expected is [text,tag]
    return tags

def getAntonymsOfWords(word):
    if(word not in vocabulary):
        return []
    setOfAntonyms = set()
    for syn in wordnet.synsets(word):
        for l in syn.lemmas():
            anton = l.antonyms()
            if(anton!=[]):
                setOfAntonyms.add(anton[0].name())
    if(len(setOfAntonyms) == 0):
        if(VERBOSE_PRINTING): print("No antonyms found for word {0}".format(word))
    return list(setOfAntonyms)

def returnReplacementsForWord(word):
    
    if(word not in vocabulary):
        print(" --- {0} not in vocabulary ---".format(word))
        return []
    possibleReplacements = [word[0] for word in word_vectors.most_similar(word,topn=NUMBER_OF_ALTERNATIVES)]
    
    if(possibleReplacements == []):
        print(" --- No replacements for word {0} ---".format(word))
    antonyms = getAntonymsOfWords(word)
    if(antonyms != []):
        possibleReplacements.extend(antonyms)
        if(VERBOSE_PRINTING): print("Some antonyms for word {0} are {1}".format(word, antonyms[:3]))
        return possibleReplacements
    return possibleReplacements
    
def posApprovedReplacements(alternativeWords, userTokens, indexOfToken):
    if(alternativeWords == []):
        return []
    tempTokens = userTokens[:]
    POSTokens = getPOSTags(tempTokens)
    validWords = []
    
    mainTag = POSTokens[indexOfToken][1]
    mainWord = userTokens[indexOfToken]
    
    for ind,word in enumerate(alternativeWords):
        tempTokens[indexOfToken] = word
        posTags = getPOSTags(tempTokens)
        newTag = (posTags[indexOfToken])[1]
        
        if(str(newTag) == str(mainTag)):
            if(VERBOSE_PRINTING): print("Word {0}[{1}] replaced with {2}[{3}]".format(mainWord, mainTag, word,newTag))
            validWords.append(word)
    if(validWords == [] and VERBOSE_PRINTING):
        print("No POS words found for word {} with tag {}".format(mainWord, mainTag));
    return validWords
    
def generateHTMLObjectsFromSentence(sentenceObj):
    
    allSentences = sentenceObj.getFinalSentences()
    indexToAlts = sentenceObj.indexToSetOfWords;
    indexToChange = list(indexToAlts.keys());
    
    listOfSentenceObjs = []
    for sentence in allSentences:
        copySentence = cleanAndTokenizeText(sentence)
        for index in indexToChange:
            copySentence[index] = cstr("[{0}]".format(copySentence[index]), "blue", italics=True);
        listOfSentenceObjs.append(SentenceWithHTML(sentence, ' '.join(copySentence)));
    
    return listOfSentenceObjs
    
    
    

## Sentence Chunking and Appending

In [6]:
def helper_combine(mainList, myList):
    '''
    helper function for CombineSentenceChunks
    '''
    newList = []
    for val in myList:
        for mainVal in mainList:
#             if(VERBOSE_PRINTING): print("Combining {0} with {1}".format(' '.join(val), ' '.join(mainVal)));
            newList.append(val + mainVal);
    return newList;

def combineSentenceChunks(wholeSentence, dictOfChunks):
    '''
        Uses the helper_combine function to generate all possible combinatios and permuations of the chunks
        and any alternatives.
        
        The logic is to use the end of the sentence, apply each possible chunk from the previous key's chunks
        to every possible chunk of this key's.
        
        The helper function is used to allow us to reuse the list of alreadyGeneratedChunks and constantly
        append to them.
        
        To understand the logic better, take a look at this gist:
        https://gist.github.com/sunnyMiglani/cf85407a9e6928237b1436cc2bc95fa4
        
    '''
    reversedKeys = sorted(dictOfChunks.keys(), reverse=True)
    completeSentences = [];
    mainArr = dictOfChunks[reversedKeys[0]]
    for ind in range(1, len(reversedKeys)):
        key = reversedKeys[ind];
        mainArr = helper_combine(mainArr, dictOfChunks[key]);
        
    return mainArr;
        
def generateSentenceChunks(wholeSentence, keyToChange, nextKey, listOfMyAlternatives):
    '''
        Generates sentence chunks by iterating through the list of alternatives
        Chunking the sentence to start from current key to next key.
        This means that the sentence always goes from key 'x' to key 'y'
        
        Example:
        "I really <hate> hot chocolate, but I <prefer> hot coffee"
        Calling generateSentenceChunks will create an example sentence:
            - "<altWordForHate> hot chocolate , but I "
        
        Remember to append the first stretch of the string to the first key's chunk for proper use!
    '''
    newList = list(listOfMyAlternatives)
    newList.append(wholeSentence[keyToChange]);
    generatedSentences = []
    for myAlt in newList:
        newSentence = wholeSentence[:]
        newSentence[keyToChange] = myAlt
        if(VERBOSE_PRINTING): print("Generated : {}".format(newSentence[keyToChange:nextKey]))
        generatedSentences.append(newSentence[keyToChange:nextKey]);
        
    return generatedSentences
    
def returnCombinationsOfStrings(sentenceObj):
    
    indexToWordDict = sentenceObj.indexToSetOfWords;
    originalSentence = sentenceObj.ogSentence;
    tokenizedSentence = cleanAndTokenizeText(originalSentence)
    reversedKeys = sorted(indexToWordDict.keys(), reverse=True)
    dictAlternatives  = {}

    sortedKeys = sorted(indexToWordDict.keys())
    sentenceChunks = {}
    htmlChunks = {}
    
    for ind in range(0,len(sortedKeys)):
        key = sortedKeys[ind]
        nextKey = sortedKeys[ind+1] if ind+1 < len(sortedKeys) else len(tokenizedSentence)
        sentenceChunks[key] = generateSentenceChunks(tokenizedSentence, key, nextKey, indexToWordDict[key])

    if(sortedKeys[0] != 0):
        newList = []
        for thislist in sentenceChunks[sortedKeys[0]]:
            newList.append(tokenizedSentence[0:sortedKeys[0]] + thislist)
        sentenceChunks[sortedKeys[0]] = newList;
        
    finalOptions = combineSentenceChunks(tokenizedSentence, sentenceChunks)
    
    finalSentences = []
    for val in finalOptions:
        sentence = ' '.join(val)
        finalSentences.append(sentence)
    
    sentenceObj.resetFinalSentences()
    sentenceObj.addFinalSentences(finalSentences)   
    return sentenceObj

In [7]:
def getAlternativeSentences(sentenceObj):
    mainSentence = sentenceObj.ogSentence;
    mainSentiment = sentenceObj.ogSentiment;

    sentenceTokens = cleanAndTokenizeText(mainSentence)

    for ind, word in enumerate(sentenceTokens):
        alternativeSentenceWithHTML = []
        copyOfTokens = sentenceTokens[:]
        replacements = []
        
        score = senty.polarity_scores(word)['compound']
        if(score != 0.0):
            tempReplacements = returnReplacementsForWord(word) # get embedding based relations
            if(tempReplacements == []):
                print("No replacements found at all for word {0}".format(word))
                continue
            replacements = posApprovedReplacements(tempReplacements[:], copyOfTokens[:], ind)
            finalReplacements = []
            if(VERBOSE_PRINTING): skippedWords = []
            for word in replacements:
                thisSentiment = senty.polarity_scores(word)['compound']
                if(thisSentiment != 0.0):
                    finalReplacements.append(word)
                else:
                    if(VERBOSE_PRINTING):
#                         print("Sentiment of Skipped word {} is {}".format(word, senty.polarity_scores(word)))
                        skippedWords.append(word)
            if(VERBOSE_PRINTING and len(skippedWords) > 0):
                print("some skipped words: {0}".format(skippedWords[:5]));
            if(finalReplacements == []):
                if(VERBOSE_PRINTING): print(" -- No POS approved words! -- for word {0}\n some non-POS:{1}".format(word, tempReplacements[:4]))
                continue
            sentenceObj.addAlternativesByIndex(ind, finalReplacements)
    return sentenceObj




In [8]:
def interactionHelper(dictOfAltObjs, sentiments, thisSent, lastWorkingIndex=0):
    if(thisSent in sentiments):
        mainIndex = sentiments.index(thisSent)
        
    else:
        mainIndex = lastWorkingIndex
    
    numLower =  mainIndex  # Mod to keep it in range 0 -> val 
    numHigher = len(sentiments) - mainIndex - 1

    print("There are totally {0} possible sentences".format(len(sentiments)))
    print("Current sentiment is at index {0}, and length of sentiments is {1}".format(mainIndex, len(sentiments)));
    print(" <--- {0} lower and {1} ---> higher ".format(numLower, numHigher))
    print("Would you like to see a lower sentiment or higher sentiment or ALL?")
    print("Please enter 1 for lower, 2 for higher or 0 for all")
    
    targetSentiment = thisSent
    textInput = input()
    if(textInput == ""):
        print("You entered nothing! exiting")
        return
        
    if(textInput.isdigit()):
        textInput = int(textInput)
        if(textInput == 0):
            if(VERBOSE_PRINTING): print("Printing sentiments from : {0}".format(sentiments))
            for sentiment in sentiments:
                obj = dictOfAltObjs[sentiment][0];
                displayText("{0}  : {1}".format(obj.getHTML(), sentiment))
            return
        elif(textInput == 1 and numLower > 0 and (mainIndex - 1) >= 0):
            targetSentiment = sentiments[mainIndex - 1]
            htmlSentence = dictOfAltObjs[targetSentiment][0].getHTML();
            displayText("{0} : {1}".format(htmlSentence, targetSentiment))
            
        elif(textInput == 2 and numHigher < len(sentiments) and (mainIndex + 1) < len(sentiments)):
            targetSentiment = sentiments[mainIndex + 1]
            htmlSentence = dictOfAltObjs[targetSentiment][0].getHTML();
            displayText("{0} : {1}".format(htmlSentence, targetSentiment))
            
        else:
            print("You entered an invalid option")
            interactionHelper(dictOfAltObjs, sentiments, thisSent, mainIndex)
            return
        
    interactionHelper(dictOfAltObjs, sentiments, targetSentiment,mainIndex)



def interactionWithUser(mainSentenceObj, dictOfAltObjs,sentiments, isUser=False):
    if(isUser == False):
        return
    else:
        mainSentence = mainSentenceObj.ogSentence;
        mainSent = mainSentenceObj.ogSentiment;
        if(VERBOSE_PRINTING):
            print("The following are the sentiment values for the input")
            for ind,sent in enumerate(sentiments):
                print("{0} - {1}".format(ind+1,sent))
        
        interactionHelper(dictOfAltObjs, sentiments, mainSent) 

In [9]:
def extractTwitterDataset():
    df_tweets = pd.read_csv( pathToDatasets + 'cleanedTweets.csv', nrows=NUM_OF_TWEETS, skiprows =TWEET_START)
    tweets = df_tweets.values
    return tweets;

def createDictionaryOfSentStrings(sentencesWithSentiment):
    dict_sentimentToStringObjects = {}
    for obj in sentencesWithSentiment:
        sent = obj.getSentiment()
        if(dict_sentimentToStringObjects.get(sent) == None):
            dict_sentimentToStringObjects[sent] = [obj]
        else:
            (dict_sentimentToStringObjects[sent]).append(obj)

    return dict_sentimentToStringObjects;

def runThroughTweets():
    tweets = extractTwitterDataset()
    
    counterOfTweets = 0;
    for counter,tweet in enumerate(tweets):
        counterOfTweets +=1;
        tweet = tweet[0]
        tweetTokens = cleanAndTokenizeText(tweet)
        mainSentiment = senty.polarity_scores(tweet)['compound']
        if(mainSentiment == 0):
            continue
        print("\n\n Tweet: {0}:{1}\n".format(tweet,mainSentiment))   
        sentenceObj = Sentence(tweet, mainSentiment)
        sentenceObj = getAlternativeSentences(sentenceObj)
        replacementDictionary = sentenceObj.getDictOfIndexWords();
        if(len(replacementDictionary) <= 0):        
            print(" -- No new Strings generated ---\n\n")
            continue

        keysToChange = replacementDictionary.keys();
        for key in keysToChange:
            if(VERBOSE_PRINTING):print("{0}'th word's ({2}) options: {1}".format(key,replacementDictionary[key],tweetTokens[key]))

        sentenceObj = returnCombinationsOfStrings(sentenceObj)
        sentencesWithSentiment = printStrings(sentenceObj)
       
        dict_sentimentToStringObjects = createDictionaryOfSentStrings(sentencesWithSentiment)
        
        sentiments = sorted(dict_sentimentToStringObjects.keys())
        if(VERBOSE_PRINTING):print("sentiments:{0}".format(sentiments))
        
    return counterOfTweets;
        
numOfTweets = runThroughTweets()
print("num of tweets done : {0}".format(numOfTweets))



 Tweet: is upset that he can not update his facebook by texting it... and might cry as a result  school today also. blah!:-0.75

No antonyms found for word upset
Word upset[VBN] replaced with shocked[VBN]
Some antonyms for word cry are ['laugh']
Word cry[VB] replaced with scream[VB]
Word cry[VB] replaced with shout[VB]
Word cry[VB] replaced with laugh[VB]
some skipped words: ['shout']
No antonyms found for word blah
Word blah[UH] replaced with goy[UH]
Word blah[UH] replaced with vich[UH]
some skipped words: ['goy', 'vich']
 -- No POS approved words! -- for word vich
 some non-POS:['goy', 'stuh', 'blagojevich', 'vich']
1'th word's (upset) options: {'shocked'}
15'th word's (cry) options: {'scream', 'laugh'}
Generated : ['shocked', 'that', 'he', 'can', 'not', 'update', 'his', 'facebook', 'by', 'texting', 'it', '...', 'and', 'might']
Generated : ['upset', 'that', 'he', 'can', 'not', 'update', 'his', 'facebook', 'by', 'texting', 'it', '...', 'and', 'might']
Generated : ['scream', 'as', 'a

sentiments:[-0.75, -0.7263, -0.7177, -0.69, 0.2244, 0.2942]


 Tweet:  i dived many times for the ball. managed to save 50  the rest go out of bounds:0.4939

No antonyms found for word save
Word save[VB] replaced with help[VB]
Word save[VB] replaced with effort[VB]
some skipped words: ['effort']
10'th word's (save) options: {'help'}
Generated : ['help', '50', 'the', 'rest', 'go', 'out', 'of', 'bounds']
Generated : ['save', '50', 'the', 'rest', 'go', 'out', 'of', 'bounds']


sentiments:[0.4019, 0.4939]


 Tweet: my whole body feels itchy and like its on fire :-0.25

No antonyms found for word itchy
Word itchy[JJ] replaced with scratchy[JJ]
Word itchy[JJ] replaced with runny[JJ]
some skipped words: ['scratchy', 'runny']
 -- No POS approved words! -- for word runny
 some non-POS:['scratchy', 'itching', 'blisters', 'rashes']
Some antonyms for word like are ['unalike', 'dislike', 'unlike']
Word like[IN] replaced with unlike[IN]
some skipped words: ['unlike']
 -- No POS approved words! -- for word unlike
 some non-POS:['such', 'even', 'you', '?']
Some antonyms for word fire are ['hire']
Word fire[NN] replaced with blaze[NN]
Word fire[NN] replaced with firing[NN]
Word fire[NN] replaced with hire[NN]
some skipped words: ['blaze', 'hire']
9'th word's (fire) options: {'firing'}
Generated : ['firing']
Generated : ['fire']


sentiments:[-0.25]


 Tweet:  no it is not behaving at all. i am mad. why am i here because i can not see you all over there. :-0.6597

Some antonyms for word no are ['yes', 'all']
Word no[UH] replaced with yes[UH]
No antonyms found for word mad
Word mad[JJ] replaced with bovine[JJ]
Word mad[JJ] replaced with bse[JJ]
Word mad[JJ] replaced with spongiform[JJ]
Word mad[JJ] replaced with crazy[JJ]
some skipped words: ['bovine', 'bse', 'spongiform']
0'th word's (no) options: {'yes'}
10'th word's (mad) options: {'crazy'}
Generated : ['yes', 'it', 'is', 'not', 'behaving', 'at', 'all', '.', 'i', 'am']
Generated : ['no', 'it', 'is', 'not', 'behaving', 'at', 'all', '.', 'i', 'am']
Generated : ['crazy', '.', 'why', 'am', 'i', 'here', 'because', 'i', 'can', 'not', 'see', 'you', 'all', 'over', 'there', '.']
Generated : ['mad', '.', 'why', 'am', 'i', 'here', 'because', 'i', 'can', 'not', 'see', 'you', 'all', 'over', 'there', '.']


sentiments:[-0.6597, -0.5574, -0.128, 0.0772]


 Tweet: need a hug :0.4767

No antonyms found for word hug
Word hug[NN] replaced with goodbye[NN]
Word hug[NN] replaced with kiss[NN]
some skipped words: ['goodbye']
2'th word's (hug) options: {'kiss'}
Generated : ['kiss']
Generated : ['hug']


...

sentiments:[-0.6249, -0.6124, -0.5719, 0.6369]


 Tweet: just going to cry myself to sleep after watching marley and me.  :-0.4767

Some antonyms for word cry are ['laugh']
Word cry[VB] replaced with scream[VB]
Word cry[VB] replaced with shout[VB]
Word cry[VB] replaced with laugh[VB]
some skipped words: ['shout']
3'th word's (cry) options: {'scream', 'laugh'}
Generated : ['scream', 'myself', 'to', 'sleep', 'after', 'watching', 'marley', 'and', 'me', '.']
Generated : ['laugh', 'myself', 'to', 'sleep', 'after', 'watching', 'marley', 'and', 'me', '.']
Generated : ['cry', 'myself', 'to', 'sleep', 'after', 'watching', 'marley', 'and', 'me', '.']


sentiments:[-0.4767, -0.4019, 0.5574]


 Tweet: im sad now  miss.lilly:-0.4767

Some antonyms for word sad are ['glad']
Word sad[VBP] replaced with sorry[VBP]
Word sad[VBP] replaced with tragic[VBP]
Word sad[VBP] replaced with happy[VBP]
Word sad[VBP] replaced with unfortunate[VBP]
Word sad[VBP] replaced with terribly[VBP]
Word sad[VBP] replaced with glad[VBP]
1'th word's (sad) options: {'tragic', 'sorry', 'terribly', 'happy', 'unfortunate', 'glad'}
Generated : ['tragic', 'now', 'miss.lilly']
Generated : ['sorry', 'now', 'miss.lilly']
Generated : ['terribly', 'now', 'miss.lilly']
Generated : ['happy', 'now', 'miss.lilly']
Generated : ['unfortunate', 'now', 'miss.lilly']
Generated : ['glad', 'now', 'miss.lilly']
Generated : ['sad', 'now', 'miss.lilly']


sentiments:[-0.5574, -0.4767, -0.4588, -0.0772, 0.4588, 0.5719]


 Tweet: ooooh.... lol  that leslie.... and ok i will not do it again so leslie will not  get mad again :0.7669

No antonyms found for word lol
Word lol[VB] replaced with coxhill[VB]
some skipped words: ['coxhill']
 -- No POS approved words! -- for word coxhill
 some non-POS:['choua', 'coxhill', 'creme', '10cc']
No antonyms found for word ok
Word ok[UH] replaced with okay[UH]
Word ok[UH] replaced with yeah[UH]
No antonyms found for word mad
Word mad[JJ] replaced with bovine[JJ]
Word mad[JJ] replaced with crazy[JJ]
some skipped words: ['bovine']
9'th word's (ok) options: {'yeah', 'okay'}
21'th word's (mad) options: {'crazy'}
Generated : ['yeah', 'i', 'will', 'not', 'do', 'it', 'again', 'so', 'leslie', 'will', 'not', 'get']
Generated : ['okay', 'i', 'will', 'not', 'do', 'it', 'again', 'so', 'leslie', 'will', 'not', 'get']
Generated : ['ok', 'i', 'will', 'not', 'do', 'it', 'again', 'so', 'leslie', 'will', 'not', 'get']
Gener

sentiments:[0.6943, 0.7215, 0.7452, 0.7669]


 Tweet: meh... almost lover is the exception... this track gets me depressed every time. :0.0534

No antonyms found for word meh
Word meh[NN] replaced with lehs[NN]
some skipped words: ['lehs']
 -- No POS approved words! -- for word lehs
 some non-POS:['stee', 'kahl', 'tehr', 'lahl']
No antonyms found for word lover
Word lover[NN] replaced with boyfriend[NN]
Word lover[NN] replaced with girlfriend[NN]
Word lover[NN] replaced with friend[NN]
Word lover[NN] replaced with mistress[NN]
some skipped words: ['boyfriend', 'girlfriend', 'mistress']
Some antonyms for word depressed are ['elate']
Word depressed[JJ] replaced with weak[JJ]
Word depressed[JJ] replaced with distressed[JJ]
Word depressed[JJ] replaced with insecure[JJ]
Word depressed[JJ] replaced with worried[JJ]
Word depressed[JJ] replaced with disturbed[JJ]
3'th word's (lover) options: {'friend'}
12'th word's (depressed) options: {'insecure', 'worried', 'disturbed', 'weak', 'distressed'}

sentiments:[-0.1761, -0.0754, -0.0498, -0.024, 0.0018, 0.079, 0.1045, 0.1548, 0.2516]


 Tweet: some1 hacked my account on aim  now i have to make a new one:-0.4019

No antonyms found for word hacked
Word hacked[VBD] replaced with bludgeoned[VBD]
Word hacked[VBD] replaced with mutilated[VBD]
Word hacked[VBD] replaced with butchered[VBD]
some skipped words: ['bludgeoned', 'mutilated', 'butchered']
 -- No POS approved words! -- for word butchered
 some non-POS:['machetes', 'bludgeoned', 'mutilated', 'hacking']
 -- No new Strings generated ---


num of tweets done : 30


## Testing Code


In [10]:
specificWord = "good"
def testOneWord(word=""):
    if(word==""):
        return
    print(word)

In [ ]:
specificString = ""
def specificString(textString=""):
    if(textString == "" or textString == None):
        return
    textString = dataClean(textString)
    mainSentiment = senty.polarity_scores(textString)['compound']
    if(mainSentiment == 0):
        print("{} \n No sentiment found in sentence".format(textString));
        return;
    print("\n {0}:{1}\n".format(textString,mainSentiment))   
    sentenceObj = Sentence(textString, mainSentiment)
    sentenceObj = getAlternativeSentences(sentenceObj)
    replacementDictionary = sentenceObj.getDictOfIndexWords();
    if(len(replacementDictionary) <= 0):        
        print(" -- No new Strings generated ---\n\n")
        return
    
    keysToChange = replacementDictionary.keys();
    if(VERBOSE_PRINTING):
        for key in keysToChange:
            print("{0}'th word's options: {1}".format(key,replacementDictionary[key]))
        
    sentenceObj = returnCombinationsOfStrings(sentenceObj)
    allPossibleSentences = printStrings(sentenceObj)
    
    dict_sentimentToStringObjects = createDictionaryOfSentStrings(allPossibleSentences)
        
    sentiments = sorted(dict_sentimentToStringObjects.keys())

    
    interactionWithUser(sentenceObj, dict_sentimentToStringObjects, sentiments[0:10], True);

print("Enter a string to use")


inputText = input()
if(inputText == ""):
    return
else:
    specificString(inputText)

Enter a string to use
